### Pyspark Dataframes continued, handling missing values, SQL library

Jay Urbain, PhD   
3/15/2024

- Dropping Columns
- Dropping Rows
- Various Parameter In Dropping functionalities
- Handling Missing values by Mean, MEdian And Mode
- Updating a dataframe

PySpark documentation:   
https://spark.apache.org/docs/latest/api/python/reference/index.html 

Answer the TODO items.

In [ ]:
!python --version

In [ ]:
# option 1

import os

JAVA_HOME = "/home/jay/anaconda3/envs/pyspark"
os.environ["JAVA_HOME"] = JAVA_HOME

# option 2

import sys
sys.path.append("/Users/jayurbain/opt/miniconda3/envs/pyspark") 

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [ ]:
df_pyspark=spark.read.csv('test2.csv',header=True,inferSchema=True)

In [ ]:
df_pyspark.printSchema()

In [ ]:
df_pyspark.show()

In [ ]:
# drop the name column

df_pyspark.drop('Name').show()

In [ ]:
df_pyspark.show()

In [ ]:
df_pyspark.na.drop().show()

In [ ]:
# any==how
df_pyspark.na.drop(how="any").show()

In [ ]:
# threshold

df_pyspark.na.drop(how="any",thresh=3).show()

In [ ]:
## You can use subset for selecting rows

df_pyspark.na.drop(how="any",subset=['Age']).show()

In [ ]:
### Filling the Missing Value
df_pyspark.na.fill('Missing Values',['Experience','age']).show()

In [ ]:
df_pyspark.show()

TODO: What's goig on here? I thought we removed those rows. Explain here:




In [ ]:
df_pyspark.printSchema()

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age', 'Experience', 'Salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("median")

In [ ]:
# Add imputation cols to df
imputer.fit(df_pyspark).transform(df_pyspark).show()

Update Spark Dataframe

You can do an update on a PySpark DataFrame Column using withColum(), select() and sql(), since DataFrame’s are distributed immutable collection you can’t really change the column values however when you change the value using withColumn() or any approach, PySpark returns a new Dataframe with updated values. 

Read data into dataframe from list

In [ ]:
!python --version

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

data = [('James','Smith','M',3000), ('Anna','Rose','F',4100),
  ('Robert','Williams','M',6200)
]
columns = ["firstname","lastname","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()


Below PySpark code updates the salary column value of DataFrame by multiplying salary by 3 times. 

Note that withColumn() is used to update or add a new column to the DataFrame, when you pass the existing column name to the first argument to withColumn() operation it updates it , if the value is new then it creates a new column.

In [ ]:
df2=df.withColumn("salary", df.salary*3)
df2.show()


Update Column Based on Condition

Update a column value based on a condition by using `When Otherwise`. The example below updates the gender column with a value Male for M, Female for F and keeps the same value for others.

In [ ]:
from pyspark.sql.functions import when
df3 = df.withColumn("gender", when(df.gender == "M","Male") \
      .when(df.gender == "F","Female") \
      .otherwise(df.gender))
df3.show()


Update DataFrame Column Data Type

You can also update a Data Type of a column using withColumn(). In addition you have to use the cast() function of the PySpark Column class. The code below updates salary column to String type.

PySpark SQL Update

In [ ]:
df.createOrReplaceTempView("PER")
df5=spark.sql("select firstname,gender,salary*3 as salary from PER")
df5.show()


#### TODO: Use spark.sql to order the original dataframe by salary descending

#### TODO: Use spark.sql to determine the average salary for each gender, order results by average salary descending